# Splitting File
In this notebook, we will read the HIS data fike which contain 4 different tables in one CSV file. We will split them into 4 different tables and ensure all columns names are correct.

In [62]:
import polars as pl
import os
import json

Read the big file and create new directory for new files

In [63]:
# Define file paths
big_csv_path = "C:/Users/reema.alhenaki/Desktop/llama3_Data/data/raw/sample_data_vida.csv"
output_folder = "C:/Users/reema.alhenaki/Desktop/llama3_Data/data/split"
os.makedirs(output_folder, exist_ok=True)

In [64]:
# Read the big CSV fully
df = pl.read_csv(big_csv_path)

In polars, the suffix '_duplicated' is added when it is spliting a file. This function helps to remove the suffix to ensure all columns names are correct and reflects the original table.

In [65]:
# Function to clean '_duplicated' suffixes
def clean_duplicated_columns(df: pl.DataFrame) -> pl.DataFrame:
    """
    Remove '_duplicated', '_duplicated_0', '_duplicated_1', etc. suffixes from column names.
    """
    new_columns = []
    for col in df.columns:
        clean_col = re.sub(r'_duplicated(_\d+)?$', '', col)
        new_columns.append(clean_col)
    return df.rename({old: new for old, new in zip(df.columns, new_columns)})


In [66]:
# Helper function to convert Excel column letters to index
def excel_col_to_index(col):
    col = col.upper()
    index = 0
    for i, c in enumerate(reversed(col)):
        index += (ord(c) - ord('A') + 1) * (26 ** i)
    return index - 1  # zero based index

In [67]:
# Get all column names
all_columns = df.columns

# Define your ranges
patient_start = excel_col_to_index('A')
patient_end = excel_col_to_index('CR')
vital_start = excel_col_to_index('CS')
vital_end = excel_col_to_index('FA')
appoint_start = excel_col_to_index('FB')
appoint_end = excel_col_to_index('IW')
doctor_start = excel_col_to_index('IX')
doctor_end = excel_col_to_index('JP')


In [68]:
# Extract columns for each table
patient_cols = all_columns[patient_start:patient_end+1]
vital_cols = all_columns[vital_start:vital_end+1]
appoint_cols = all_columns[appoint_start:appoint_end+1]
doctor_cols = all_columns[doctor_start:doctor_end+1]

In [69]:
# Clean duplicated suffixes AFTER splitting
df_patient = clean_duplicated_columns(df_patient)
df_vital = clean_duplicated_columns(df_vital)
df_appoint = clean_duplicated_columns(df_appoint)
df_doctor = clean_duplicated_columns(df_doctor)

In [70]:
# Save each to CSV
df_patient.write_csv(os.path.join(output_folder, "HIS_Patient.csv"))
df_vital.write_csv(os.path.join(output_folder, "HIS_PatientVitalSigns.csv"))
df_appoint.write_csv(os.path.join(output_folder, "HIS_Appointment.csv"))
df_doctor.write_csv(os.path.join(output_folder, "HIS_DoctorOrder.csv"))

print("Files saved successfully.")


Files saved successfully.
